## 0 - Introduction

Once the `bidsmap.yaml` is created and properly tested,
the bidsification will be performed easely,
just running the `bidsme.bidsify()` function.

But first we need to initialize the paths to deataset,
load helper fonctions from `tools.py` and initialize the bidsme:

In [ ]:
%run "./Configuration/00-tutorial-paths.py"

In [ ]:
%run "../Tools/tools.py"

In [ ]:
import bidsme
logger = bidsme.init()

## 1 - Bidsification

The function `bidsme.bidsify()` is responsible for the actual bidsification
of the dataset:

In [ ]:
help(bidsme.bidsify)

The parameters for `bidsme.bidsify()` are the same as `bidsme.mapper()`:

In [ ]:
bidsme bidsify $PREPARED_PATH $BIDSIFIED_PATH -b $MAP_FILE

The `bidsify` will produce some warnings, but if `bidsmap` file from previous tutorial 
is consructed correctly, it should no be errors.

The first two warnings you may encounter is:
```
bidsify(203) - WARNING Dataset description file 'dataset_description.json' not found in '/home/beliy/Work/bidsme/bidsme_example/example1/bids'
bidsify(209) - WARNING Dataset readme file 'README' not found in '/home/beliy/Work/bidsme/bidsme_example/example1/bids'
```

[BIDS](https://bids-specification.readthedocs.io/en/stable/03-modality-agnostic-files.html#dataset-description)
requires two description files `README` and `dataset_description.json` to be
in the datset, so bidsme warns user if these files are missing.

Next warning you may encounter is related to diffusion images:
```
Modules.MRI.MRI(78) - WARNING 013-cmrr_mbep2d_diff_NODDI/0 missing bvec file for diffusion recording
```

[BIDS](https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/01-magnetic-resonance-imaging-data.html#required-gradient-orientation-information)
requires that each diffusion file is accompagnied by a gradient orientation file.
If bidsme encounter a diffusion file (file that will go into `dwi` modality),
it will check if there a file with same name but with extentions `.bval` and `.bvec`.
If these files are found, they will inherit same bidsified name as diffusion image file
and copied to bidsified dataset, but if these files are missing, an warning will be shown.

These bval/bvec files must be created by user, either using script, or using plugin.

### 1-Verifying bidsificated names

After the first bidsification it's important to go into bidsified folder and verify
manually the bidsified names, by comparing them with [projected names](./Files/projected_names.md):

In [ ]:
!ls $BIDSIFIED_PATH/sub-001/ses-HCL/fmap

You may see slight deviation from the original names, namely apparation of the `run-` entity, but overall the names should follow the initial schema.

If one of the names are incorrect, you need to find the source of error in `bidsmap.yaml`
and correct it. Once it's done, remove bidsified dataset, and run bidsification again,
untill all names are as they should.

### 2-Conflict in the bidsified names   

`Bidsme` will never overwrite existing files during bidsification.
If he tries to bidsify a given file, and that file already exists
in the bidsify dataset, `bidsme` will refuse to overwrite the file and raise 
an **error**:
```
bidsify(107) - ERROR bids/sub-004/ses-s01629/fmap/sub-004_ses-s01629_acq-RFoff_dir-AP_run-001_epi.json exists at destination
```

> *N.B.*: You can easely see this error if you run two bidsifications
steps one after another.
As it will try to bidsify the same files, that will end up
with the same names.

Normally majority of sources of duplicated file names should be automatically
detected during the mapping stage.
However it may happen that some duplicated files will escape the tests.
If it happens, you should investigate the source of conflict and fix it
as described in [basic-mapping tutorial](02-basic-mapping.ipynb).

`bidsme` will rise an error for each already processed file, which
can cause problems when new subjects into already existing dataset.

`bidsme` provide several options to control which subjects and sessions are to bidsify.
These options are the same as described in **Running on subset of subjects**
[section of basic preparation tutorial](01-basic-preparation.ipynb).

To no repeat the parameters:

 - `sub_list` with provided list of subject will make bidsification run only on select subjects
 - `sub_skip_tsv` with value `True` will run bidsification only on subjects **not present** in `participants.tsv` table in bidsified dataset
 - `sub_skip_dir` with value `True` will run bidsification only on subjects **not present** on disk, i.e. if `sub-123` exist in bidsified folder, then this subjects is skipped
 - `ses_skip_dir` with value `True` will run bidsification only on sessions **not present** on disk, similarly to `sub_skip_dir`, but at session level.
 
> *N.B.* By default it is better to run bidsification with `sub_skip_tsv`, and when needed to re-bidsify a given subject, just remove corresponding `sub-` folder and removing corresponding line in `participants.tsv`

### 3-Verification of sidecar json contents   

Outside the bidsified names it is important to verify the corectness
of values entered in the sidecar json files.
An incorrect entry may lead to incorrect image processing/reconstruction.

Normally, majority of the entries are filled automatically,
exported directly from the header
(**that's doesn'n mean they are correct! Error can happen on every level!**)
but some may be calculated in plugin or imputed by user
directly. Once you bidsify one subject, you should verify every
sidecar json for the values.
If one subject is correct, other should be correct also.